# 文本预处理
sec_text_preprocessing

对于序列数据处理问题，我们在 sec_sequence中
评估了所需的统计工具和预测时面临的挑战。
这样的数据存在许多种形式，文本是最常见例子之一。
例如，一篇文章可以被简单地看作一串单词序列，甚至是一串字符序列。
本节中，我们将解析文本的常见预处理步骤。
这些步骤通常包括：

1. 将文本作为字符串加载到内存中。
1. 将字符串拆分为词元（如单词和字符）。
1. 建立一个词表，将拆分的词元映射到数字索引。
1. 将文本转换为数字索引序列，方便模型操作。


In [29]:
import collections
import re
import jieba
from d2l import torch as d2l

## 读取数据集

首先，我们从H.G.Well的[时光机器](https://www.gutenberg.org/ebooks/35)中加载文本。
这是一个相当小的语料库，只有30000多个单词，但足够我们小试牛刀，
而现实中的文档集合可能会包含数十亿个单词。
下面的函数(**将数据集读取到由多条文本行组成的列表中**)，其中每条文本行都是一个字符串。
为简单起见，我们在这里忽略了标点符号和字母大写。


In [ ]:
def load_ai_data():
    d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt',
                                    '090b5e7e70c295757f55df93cb0a180b9691891a')

    pass

In [ ]:
# 加载训练数据 英文小说
load_ai_data()

In [ ]:
with open("/mnt/325E98425E980131/ai_data/NLP_data/贾平凹-山本.txt", 
          'r', encoding='utf8') as f:
    lines = f.readlines()
    pass


# 加载停用词表
stopwords_file = '/mnt/325E98425E980131/ai_data/NLP_data/stopwords.txt'
with open(stopwords_file, "r") as words:
    stopwords = [i.strip() for i in words]
    pass

stopwords.extend(['n', '.','（','）','-','——','(',')',' ','，'])
textList = jieba.lcut(str(lines))# .split()

q_cut_list = [i for i in textList if i not in stopwords] # 去除停用词
q_cut_list

In [129]:

# 加载英文小说
def read_time_machine(): 
    """
    将时间机器数据集加载到文本行的列表中
    """
    with open(d2l.download('time_machine'), 'r') as f:
        lines = f.readlines()
        pass
    return [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines]


# 加载汉字小说
def read_time_machine_v2(): 
    """
    将时间机器数据集加载到文本行的列表中
    """
    with open("/mnt/325E98425E980131/ai_data/NLP_data/贾平凹-山本.txt", 
              'r', encoding='utf8') as f:
        lines = f.readlines()
        pass
    
    # 加载停用词表
    stopwords_file = '/mnt/325E98425E980131/ai_data/NLP_data/stopwords.txt'
    with open(stopwords_file, "r") as words:
        stopwords = [i.strip() for i in words]
        pass
    
    stopwords.extend(['n', '.','（','）','-','——','(',')',' ','，'])
    textList = jieba.lcut(str(lines))
    # q_cut_str = " ".join(textList)
    
    q_cut_list = [i for i in textList if i not in stopwords] # 去除停用词
    return q_cut_list

In [130]:
# lines = read_time_machine()
lines = read_time_machine_v2()

print(f'# 文本总行数: {len(lines)}')
print(lines[0])
print(lines[10])

# 文本总行数: 134938
文前
著


## 词元化

下面的`tokenize`函数将文本行列表（`lines`）作为输入，
列表中的每个元素是一个文本序列（如一条文本行）。
[**每个文本序列又被拆分成一个词元列表**]，*词元*（token）是文本的基本单位。
最后，返回一个由词元列表组成的列表，其中的每个词元都是一个字符串（string）。


In [131]:
def tokenize(lines, token='word'):  #@save
    """
    将文本行拆分为单词或字符词元
    """
    if token == 'word':
        return [line for line in lines]

    elif token == 'char':
        return [list(line) for line in lines]
    
    else:
        print('错误：未知词元类型：' + token)

In [132]:
tokens = tokenize(lines, token='char')

for i in range(30):
    print(tokens[i])
    pass

['文', '前']
['辅', '文']
['作', '者', '简', '介']
['图', '书']
['版']
['编', '目']
['C', 'I', 'P']
['数', '据']
['山', '本']
['贾', '平', '凹']
['著']
['北', '京']
['人', '民', '文', '学', '出', '版', '社']
['2', '0', '1', '8']
['I', 'S', 'B', 'N']
['9', '7', '8']
['0', '1', '3', '9', '7', '8']
['Ⅰ']
['山']
['Ⅱ']
['贾']
['长', '篇', '小', '说']
['中', '国']
['当', '代']
['Ⅳ']
['I', '2', '4', '7', '.', '5']
['中', '国']
['版', '本']
['图', '书', '馆']
['C', 'I', 'P']


## 词表

词元的类型是字符串，而模型需要的输入是数字，因此这种类型不方便模型使用。
现在，让我们[**构建一个字典，通常也叫做*词表*（vocabulary），
用来将字符串类型的词元映射到从$0$开始的数字索引中**]。
我们先将训练集中的所有文档合并在一起，对它们的唯一词元进行统计，
得到的统计结果称之为*语料*（corpus）。
然后根据每个唯一词元的出现频率，为其分配一个数字索引。
很少出现的词元通常被移除，这可以降低复杂性。
另外，语料库中不存在或已删除的任何词元都将映射到一个特定的未知词元“&lt;unk&gt;”。
我们可以选择增加一个列表，用于保存那些被保留的词元，
例如：填充词元（“&lt;pad&gt;”）；
序列开始词元（“&lt;bos&gt;”）；
序列结束词元（“&lt;eos&gt;”）。


In [133]:
class Vocab:
    """
    文本词表
    """
    
    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):
        if tokens is None:
            tokens = []
            
        if reserved_tokens is None:
            reserved_tokens = []
            
        # 按出现频率排序
        counter = count_corpus(tokens)
        self._token_freqs = sorted(counter.items(), 
                                   key=lambda x: x[1],
                                   reverse=True
                                  )
        # 未知词元的索引为0
        self.idx_to_token = ['<unk>'] + reserved_tokens
        self.token_to_idx = {token: idx
                             for idx, token in enumerate(self.idx_to_token)
                            }
        
        for token, freq in self._token_freqs:
            if freq < min_freq:
                break
            if token not in self.token_to_idx:
                self.idx_to_token.append(token)
                self.token_to_idx[token] = len(self.idx_to_token) - 1

    def __len__(self):
        return len(self.idx_to_token)

    def __getitem__(self, tokens):
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]

    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]

    @property
    def unk(self):  # 未知词元的索引为0
        return 0

    @property
    def token_freqs(self):
        return self._token_freqs

    
def count_corpus(tokens):  #@save
    """
    统计词元的频率
    """
    # 这里的tokens是1D列表或2D列表
    if len(tokens) == 0 or isinstance(tokens[0], list):
        # 将词元列表展平成一个列表
        tokens = [token for line in tokens for token in line]
        
    return collections.Counter(tokens)

我们首先使用时光机器数据集作为语料库来[**构建词表**]，然后打印前几个高频词元及其索引。


In [139]:
vocab = Vocab(tokens)

print(list(vocab.token_to_idx.items())[:300])

[('<unk>', 0), ('说', 1), ('一', 2), ('子', 3), ('人', 4), ('井', 5), ('宗', 6), ('里', 7), ('秀', 8), ('上', 9), ('陆', 10), ('来', 11), ('不', 12), ('长', 13), ('菊', 14), ('头', 15), ('0', 16), ('天', 17), ('生', 18), ('个', 19), ('家', 20), ('成', 21), ('下', 22), ('杨', 23), ('县', 24), ('门', 25), ('花', 26), ('山', 27), ('三', 28), ('镇', 29), ('着', 30), ('回', 31), ('老', 32), ('保', 33), ('吃', 34), ('走', 35), ('出', 36), ('柜', 37), ('掌', 38), ('两', 39), ('剩', 40), ('事', 41), ('口', 42), ('地', 43), ('钟', 44), ('大', 45), ('陈', 46), ('十', 47), ('这', 48), ('安', 49), ('到', 50), ('好', 51), ('跑', 52), ('死', 53), ('枪', 54), ('有', 55), ('队', 56), ('杜', 57), ('得', 58), ('打', 59), ('城', 60), ('开', 61), ('先', 62), ('阮', 63), ('手', 64), ('麻', 65), ('边', 66), ('去', 67), ('五', 68), ('院', 69), ('鲁', 70), ('身', 71), ('声', 72), ('没', 73), ('想', 74), ('过', 75), ('水', 76), ('树', 77), ('周', 78), ('茶', 79), ('3', 80), ('住', 81), ('丞', 82), ('马', 83), ('发', 84), ('问', 85), ('是', 86), ('前', 87), ('看', 88), ('团', 89), ('黑', 90), ('话'

现在，我们可以(**将每一条文本行转换成一个数字索引列表**)。


In [135]:
for i in [0, 10]:
    print('文本:', tokens[i])
    print('索引:', vocab[tokens[i]])

文本: ['文', '前']
索引: [280, 87]
文本: ['著']
索引: [2852]


## 整合所有功能

在使用上述函数时，我们[**将所有功能打包到`load_corpus_time_machine`函数中**]，
该函数返回`corpus`（词元索引列表）和`vocab`（时光机器语料库的词表）。
我们在这里所做的改变是：

1. 为了简化后面章节中的训练，我们使用字符（而不是单词）实现文本词元化；
1. 时光机器数据集中的每个文本行不一定是一个句子或一个段落，还可能是一个单词，因此返回的`corpus`仅处理为单个列表，而不是使用多词元列表构成的一个列表。


In [136]:
def load_corpus_time_machine(max_tokens=-1):  #@save
    """
    返回时光机器数据集的词元索引列表和词表
    """
    # lines = read_time_machine()
    lines = read_time_machine_v2()
    tokens = tokenize(lines, 'char')
    vocab = Vocab(tokens)
    
    # 因为时光机器数据集中的每个文本行不一定是一个句子或一个段落，
    # 所以将所有文本行展平到一个列表中
    corpus = [vocab[token] for line in tokens for token in line]
    if max_tokens > 0:
        corpus = corpus[:max_tokens]
        pass
    
    return corpus, vocab

In [137]:

# 词元索引列表
corpus, vocab = load_corpus_time_machine()


len(corpus), len(vocab)

(249243, 3575)

## 小结

* 文本是序列数据的一种最常见的形式之一。
* 为了对文本进行预处理，我们通常将文本拆分为词元，构建词表将词元字符串映射为数字索引，并将文本数据转换为词元索引以供模型操作。